In [2]:
import pandas as pd
df=pd.read_csv('../champ_result/all_beta_normalized.csv')
df

,Unnamed: 0,1,2,3,4,5,6,7,8,9,...,881,882,883,884,885,886,887,888,889,890
0,cg00000957,0.825079,0.825079,0.836188,0.836188,0.855953,0.855953,0.856379,0.856379,0.833668,...,0.901550,0.901550,0.873928,0.873928,0.820407,0.820407,0.880965,0.880965,0.866919,0.866919
1,cg00001349,0.690023,0.690023,0.802989,0.802989,0.744400,0.744400,0.826541,0.826541,0.683470,...,0.864090,0.864090,0.878295,0.878295,0.699745,0.699745,0.851946,0.851946,0.784683,0.784683
2,cg00001583,0.095879,0.095879,0.030527,0.030527,0.058828,0.058828,0.103293,0.103293,0.054348,...,0.788893,0.788893,0.566003,0.566003,0.648568,0.648568,0.755152,0.755152,0.503848,0.503848
3,cg00002028,0.037414,0.037414,0.028130,0.028130,0.036667,0.036667,0.026973,0.026973,0.032372,...,0.053580,0.053580,0.037539,0.037539,0.063686,0.063686,0.067983,0.067983,0.053181,0.053181
4,cg00002837,0.393330,0.393330,0.278496,0.278496,0.354795,0.354795,0.371494,0.371494,0.372948,...,0.799111,0.799111,0.654631,0.654631,0.785799,0.785799,0.709136,0.709136,0.452765,0.452765
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349415,cg27656573,0.950538,0.950538,0.950720,0.950720,0.955993,0.955993,0.956690,0.956690,0.947600,...,0.960415,0.960415,0.965732,0.965732,0.966887,0.966887,0.971332,0.971332,0.967507,0.967507
349416,cg27657363,0.959770,0.959770,0.935633,0.935633,0.956899,0.956899,0.947483,0.947483,0.956285,...,0.903175,0.903175,0.956787,0.956787,0.949065,0.949065,0.932598,0.932598,0.887922,0.887922
349417,cg27657537,0.071075,0.071075,0.051070,0.051070,0.052173,0.052173,0.082928,0.082928,0.097815,...,0.318862,0.318862,0.358653,0.358653,0.223531,0.223531,0.232292,0.232292,0.107879,0.107879
349418,cg27662611,0.044393,0.044393,0.038378,0.038378,0.051068,0.051068,0.061392,0.061392,0.050559,...,0.083533,0.083533,0.056394,0.056394,0.073767,0.073767,0.086748,0.086748,0.029125,0.029125


In [3]:
# change the column[0] name of df
df.rename(columns={'Unnamed: 0':'ID'}, inplace=True)

In [4]:
#[0~349419,1~94]>>normal
normal=df.iloc[0:,1:95:2]
tumor=df.iloc[0:,95::2]
# reset the column name of TN
tumor.columns=range(0,tumor.shape[1])

normal.columns=range(0,normal.shape[1])

tumor.reset_index(drop=True, inplace=True)
normal.reset_index(drop=True, inplace=True)
tumor = tumor.T
normal = normal.T 

In [5]:
# Computing IQR
def IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3 - Q1
    upper_fence=Q3+IQR*1.5
    lower_fence=Q1-IQR*1.5
    return upper_fence,lower_fence
def no_outlier(df):
    upper_fence,lower_fence=IQR(df)
    ddf=df[(df>lower_fence)&(df<upper_fence)]
    return ddf

In [6]:
normal_out=no_outlier(normal)
tumor_out=no_outlier(tumor)

In [7]:
# ignore NaN and calculate mean
normal_avg = normal_out.mean(skipna=True, axis=0)
normal_avg.head(20)

0     0.843970
1     0.769853
2     0.099663
3     0.031946
4     0.389191
5     0.014140
6     0.337494
7     0.954143
8     0.022727
9     0.079507
10    0.119703
11    0.953453
12    0.649771
13    0.968327
14    0.867800
15    0.033570
16    0.737836
17    0.042565
18    0.060940
19    0.952767
dtype: float64

In [8]:
TMoSub=(tumor_out).subtract(normal_avg, axis = 1)
TMoSub

,0,1,2,3,4,5,6,7,8,9,...,349410,349411,349412,349413,349414,349415,349416,349417,349418,349419
0,0.004670,0.065996,0.590197,-0.005893,-0.081763,-0.001122,-0.079697,-0.006011,0.005950,-0.028631,...,0.158281,0.017059,-0.030644,-0.034561,0.069673,-0.009434,-0.028009,-0.026395,0.018056,-0.049024
1,NaN,-0.033603,-0.008181,0.003033,0.049574,0.002278,0.150208,0.011360,-0.014403,0.032522,...,-0.136428,-0.003020,-0.040136,0.007361,-0.048760,-0.013613,-0.060788,-0.030290,-0.012621,-0.096205
2,NaN,NaN,0.689951,0.018468,0.305408,-0.003757,0.414896,-0.003007,0.003083,-0.038052,...,0.072553,0.020862,-0.034318,-0.031782,-0.156906,0.000583,-0.046198,NaN,0.010067,0.039321
3,0.015237,0.076037,0.222345,0.009212,0.373555,-0.003336,0.388926,0.002294,-0.007744,-0.015890,...,0.184931,0.007765,-0.009615,-0.011553,0.531563,0.003172,-0.009732,NaN,0.052125,0.030556
4,0.044863,0.087323,0.329270,-0.005155,0.112867,-0.000998,0.227271,-0.021485,-0.004306,-0.031546,...,0.102277,0.001261,-0.023517,-0.029394,0.167761,0.014819,-0.004002,-0.023315,0.003590,0.049327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,0.057579,0.094237,0.689230,0.021634,0.409920,0.003626,-0.081653,-0.008140,0.000698,0.023547,...,0.204042,0.029615,-0.004481,-0.006987,0.558454,0.002970,-0.043708,0.228757,0.036322,0.030556
394,0.029958,0.108443,0.466340,0.005593,0.265441,0.000190,0.375588,0.024651,-0.007121,0.076108,...,0.111157,0.015552,-0.006519,0.004608,0.006846,0.008287,0.009904,0.268548,0.009183,-0.005656
395,-0.023564,-0.070108,0.548906,0.031740,0.396608,-0.002395,0.280701,0.005816,-0.006615,0.047310,...,0.148311,0.017548,-0.003801,-0.017358,0.208806,0.009442,0.002182,0.133426,0.026556,0.045064
396,0.036994,0.082093,0.655489,0.036037,0.319946,-0.000294,0.412149,0.008145,0.013679,0.042988,...,0.160019,0.018944,0.005225,-0.026106,0.436373,0.013887,-0.014285,0.142187,0.039537,0.068328


In [9]:
TMoSub_out=no_outlier(TMoSub)

In [10]:
TMoSub_mean = TMoSub_out.mean(skipna=True)
TMoSub_mean

0         0.017616
1         0.032420
2         0.316867
3         0.014177
4         0.120412
            ...   
349415    0.003831
349416   -0.013388
349417    0.038450
349418    0.003935
349419   -0.011709
Length: 349420, dtype: float64

In [11]:
# TMoSubm join df
df_NMTM = pd.DataFrame(TMoSub_mean)

df_NMTM.columns = ['dBeta']
s = df.iloc[0:,0:1].join(df_NMTM)
s

,ID,dBeta
0,cg00000957,0.017616
1,cg00001349,0.032420
2,cg00001583,0.316867
3,cg00002028,0.014177
4,cg00002837,0.120412
...,...,...
349415,cg27656573,0.003831
349416,cg27657363,-0.013388
349417,cg27657537,0.038450
349418,cg27662611,0.003935


In [12]:
import pandas as pd
DMP = pd.read_csv('../result/DMP_result_TN.csv')
DMP.rename(columns={'Unnamed: 0':'ID'}, inplace=True)
DMP

,ID,logFC,AveExpr,t,P.Value,adj.P.Val,B,Normal_AVG,Tumor_AVG,deltaBeta,...,gene,feature,cgi,feat.cgi,UCSC_CpG_Islands_Name,DHS,Enhancer,Phantom,Probe_SNPs,Probe_SNPs_10
0,cg05434287,0.339272,0.840818,45.932847,5.812994e-237,2.031176e-231,531.601646,0.537379,0.876651,0.339272,...,MAD1L1,Body,shore,Body-shore,chr7:2029116-2029326,NaN,NaN,NaN,rs78394594,NaN
1,cg14789818,0.437637,0.736742,43.735096,7.848004e-224,1.371125e-218,501.507785,0.345327,0.782964,0.437637,...,NaN,IGR,island,IGR-island,chr1:227748423-227748860,NaN,NaN,NaN,NaN,NaN
2,cg20699586,0.445777,0.749819,41.046639,1.934875e-207,2.253613e-202,463.926433,0.351124,0.796901,0.445777,...,NaN,IGR,island,IGR-island,chr1:227748423-227748860,NaN,NaN,NaN,NaN,NaN
3,cg24974704,0.328763,0.884327,40.149315,6.827086e-202,5.963801e-197,451.204801,0.590288,0.919050,0.328763,...,ZC3H3,Body,opensea,Body-opensea,NaN,True,NaN,NaN,NaN,NaN
4,cg27616751,0.225588,0.841574,37.786086,4.101731e-187,2.866454e-182,417.308246,0.639812,0.865400,0.225588,...,FIS1,3'UTR,opensea,3'UTR-opensea,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268107,cg10175910,-0.006401,0.943532,-2.074141,3.835349e-02,4.998537e-02,-5.664030,0.949257,0.942856,-0.006401,...,DENND3,Body,opensea,Body-opensea,NaN,NaN,NaN,NaN,NaN,NaN
268108,cg21782712,-0.003763,0.036329,-2.074112,3.835617e-02,4.998868e-02,-5.664089,0.039694,0.035932,-0.003763,...,WDR46,1stExon,shore,1stExon-shore,chr6:33256539-33256973,NaN,NaN,NaN,rs3130258,NaN
268109,cg18997124,-0.031416,0.732659,-2.074070,3.836013e-02,4.999364e-02,-5.664177,0.760757,0.729341,-0.031416,...,C6orf27,Body,opensea,Body-opensea,NaN,NaN,NaN,NaN,NaN,rs80087332
268110,cg11439068,0.001822,0.023460,2.074059,3.836115e-02,4.999478e-02,-5.664200,0.021830,0.023652,0.001822,...,PAK4,TSS200,island,TSS200-island,chr19:39616277-39616819,NaN,NaN,NaN,NaN,NaN


In [13]:
df_DMP = pd.merge(s, DMP, on='ID', how='inner')
df_DMP

,ID,dBeta,logFC,AveExpr,t,P.Value,adj.P.Val,B,Normal_AVG,Tumor_AVG,...,gene,feature,cgi,feat.cgi,UCSC_CpG_Islands_Name,DHS,Enhancer,Phantom,Probe_SNPs,Probe_SNPs_10
0,cg00001583,0.316867,0.310063,0.383781,14.103587,6.522917e-41,1.672099e-39,81.885343,0.106467,0.416530,...,NR5A2,Body,island,Body-island,chr1:200011400-200012055,NaN,NaN,NaN,NaN,NaN
1,cg00002028,0.014177,0.013361,0.045512,7.706099,3.461631e-14,1.669238e-13,20.918006,0.033562,0.046923,...,PINK1,1stExon,shore,1stExon-shore,chr1:20960045-20960551,NaN,NaN,high-CpG:20832533-20832876,NaN,NaN
2,cg00002837,0.120412,0.120412,0.496885,6.579923,8.029204e-11,2.925999e-10,13.317106,0.389191,0.509602,...,NaN,IGR,opensea,IGR-opensea,NaN,True,NaN,NaN,NaN,NaN
3,cg00003287,0.110206,0.112321,0.435837,6.435061,2.015026e-10,7.076284e-10,12.417794,0.335379,0.447700,...,TNNT2,5'UTR,opensea,5'UTR-opensea,NaN,True,True,NaN,NaN,NaN
4,cg00007036,0.001742,0.003062,0.954702,2.136217,3.293422e-02,4.336785e-02,-5.534063,0.951964,0.955026,...,ZNF362,Body,island,Body-island,chr1:33741774-33742381,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268107,cg27648216,0.166039,0.137249,0.501082,6.131173,1.310408e-09,4.262982e-09,10.590671,0.378329,0.515578,...,NaN,IGR,shelf,IGR-shelf,chr22:46457095-46458028,NaN,NaN,NaN,rs58662512,NaN
268108,cg27657363,-0.013388,-0.018651,0.929339,-4.886235,1.218885e-06,2.903599e-06,3.965315,0.946020,0.927369,...,NaN,IGR,shelf,IGR-shelf,chr22:39784354-39785104,NaN,NaN,NaN,NaN,NaN
268109,cg27657537,0.038450,0.094133,0.174295,6.021426,2.527125e-09,7.998298e-09,9.950762,0.090105,0.184238,...,MED15,Body,shore,Body-shore,chr22:20861679-20862252,True,NaN,NaN,NaN,NaN
268110,cg27662611,0.003935,0.005706,0.052314,2.530359,1.156614e-02,1.632780e-02,-4.620512,0.047211,0.052917,...,MAFF,TSS200,island,TSS200-island,chr22:38597785-38598988,NaN,NaN,NaN,NaN,NaN


In [15]:
# df_DMP.to_csv('DMP_no_filter_nabs.csv', index=False)
df_DMP.to_csv('../result/DMP_no_filter.csv', index=False)

In [21]:
# max
import pandas as pd

def find_max_dBeta(df):
    df_max = df[abs(df.dBeta)==abs(df.dBeta.max())]
    # df_max = df[abs(df.dBeta)==abs(df.dBeta.max())]
    return df_max.iloc[0][:]

result = pd.read_csv('../result/DMP_no_filter.csv')
result = result.groupby('gene').apply(find_max_dBeta)
result.to_csv('../result/DMP_max.csv')
import winsound
winsound.Beep(800, 800)
result



,ID,dBeta,logFC,AveExpr,t,P.Value,adj.P.Val,B,Normal_AVG,Tumor_AVG,...,gene,feature,cgi,feat.cgi,UCSC_CpG_Islands_Name,DHS,Enhancer,Phantom,Probe_SNPs,Probe_SNPs_10
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,cg03630821,0.330774,0.318907,0.731138,27.595273,1.396772e-121,2.696464e-118,266.927620,0.445913,0.764821,...,A1BG,Body,island,Body-island,chr19:58861341-58862406,NaN,NaN,NaN,rs2241788,NaN
A2BP1,cg00590139,0.380221,0.371265,0.434550,14.970774,2.348124e-45,7.765298e-44,92.060780,0.102497,0.473762,...,A2BP1,Body,shore,Body-shore,chr16:7354333-7354608,NaN,NaN,NaN,NaN,NaN
A2LD1,cg17371841,0.000270,-0.008047,0.961911,-2.581243,1.000365e-02,1.425846e-02,-4.491477,0.969108,0.961061,...,A2LD1,3'UTR,shore,3'UTR-shore,chr13:101184396-101184848,NaN,NaN,NaN,NaN,NaN
A2M,cg00134295,0.207143,0.195737,0.407578,8.779972,8.274144e-18,5.210784e-17,29.134256,0.232515,0.428252,...,A2M,TSS1500,opensea,TSS1500-opensea,NaN,NaN,True,NaN,NaN,NaN
A2ML1,cg21416544,0.102696,0.062505,0.800718,4.245460,2.410785e-05,4.914538e-05,1.108488,0.744815,0.807320,...,A2ML1,Body,opensea,Body-opensea,NaN,NaN,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11B,cg17101920,0.011458,0.007325,0.087089,2.785454,5.458948e-03,8.086285e-03,-3.948192,0.080538,0.087863,...,ZYG11B,Body,island,Body-island,chr1:53192023-53192428,NaN,NaN,NaN,NaN,NaN
ZYX,cg09704136,0.236211,0.222840,0.739851,17.463163,6.098736e-59,4.319052e-57,123.194378,0.540547,0.763387,...,ZYX,Body,shelf,Body-shelf,chr7:143077469-143079169,NaN,True,NaN,NaN,NaN
ZZEF1,cg26591066,0.297844,0.262805,0.802987,18.676116,6.356968e-66,6.538863e-64,139.204855,0.567938,0.830744,...,ZZEF1,Body,opensea,Body-opensea,NaN,NaN,NaN,NaN,NaN,NaN
